In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    confusion_matrix, roc_auc_score,
    recall_score, precision_score, f1_score
)

from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

df = pd.read_csv('/content/drive/MyDrive/final_year_project/fraud_oracle 4.csv')


In [ ]:
# Data Cleaning from Data Prep
df = df.drop(index=1516, errors="ignore")

# Replace Age = 0 with median
if "Age" in df.columns:
    median_age = df.loc[df["Age"] != 0, "Age"].median()
    df["Age"] = df["Age"].replace(0, median_age)

# Drop useless columns
df = df.drop(columns=["PolicyNumber", "RepNumber", "Year"], errors="ignore")

# Encode binary columns (
binary_columns = [c for c in df.columns if c != "FraudFound_P" and df[c].nunique(dropna=False) == 2]
print("Binary columns:", binary_columns)

le = LabelEncoder()
for col in binary_columns:
    df[col] = le.fit_transform(df[col].astype(str))
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(f"{col}: {mapping}")

Binary columns: ['AccidentArea', 'Sex', 'Fault', 'PoliceReportFiled', 'WitnessPresent', 'AgentType']
AccidentArea: {'Rural': np.int64(0), 'Urban': np.int64(1)}
Sex: {'Female': np.int64(0), 'Male': np.int64(1)}
Fault: {'Policy Holder': np.int64(0), 'Third Party': np.int64(1)}
PoliceReportFiled: {'No': np.int64(0), 'Yes': np.int64(1)}
WitnessPresent: {'No': np.int64(0), 'Yes': np.int64(1)}
AgentType: {'External': np.int64(0), 'Internal': np.int64(1)}


In [ ]:
TARGET = "FraudFound_P"



# Ordinal mapping
ORDINAL_MAPS = {
    "PastNumberOfClaims": {"none": 0, "1": 1, "2 to 4": 2, "more than 4": 3},
    "NumberOfSuppliments": {"none": 0, "1 to 2": 1, "3 to 5": 2, "more than 5": 3},
    "Days_Policy_Accident": {"none": 0, "1 to 7": 1, "8 to 15": 2, "15 to 30": 3, "more than 30": 4},
    "Days_Policy_Claim": {"none": 0, "8 to 15": 1, "15 to 30": 2, "more than 30": 3},
    "AgeOfVehicle": {"new": 0, "2 years": 1, "3 years": 2, "4 years": 3, "5 years": 4, "6 years": 5, "7 years": 6, "more than 7": 7},
    "AgeOfPolicyHolder": {"16 to 17": 0, "18 to 20": 1, "21 to 25": 2, "26 to 30": 3, "31 to 35": 4, "36 to 40": 5, "41 to 50": 6, "51 to 65": 7, "over 65": 8},
    "AddressChange_Claim": {"under 6 months": 0, "1 year": 1, "2 to 3 years": 2, "4 to 8 years": 3, "no change": 4},
    "NumberOfCars": {"1 vehicle": 1, "2 vehicles": 2, "3 to 4": 3, "5 to 8": 4, "more than 8": 5},
}

for col, mapping in ORDINAL_MAPS.items():
    if col in df.columns:
        # fillna(-1) handles unseen categories safely
        df[col] = df[col].map(mapping).fillna(-1).astype(int)

# Split into x/y
y = df[TARGET].astype(int)
X = df.drop(columns=[TARGET])

# Train/test split
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# One-hot encode any remaining object columns
obj_cols = X_train_raw.select_dtypes(include=["object"]).columns.tolist()
X_train = pd.get_dummies(X_train_raw, columns=obj_cols, drop_first=False)
X_test  = pd.get_dummies(X_test_raw,  columns=obj_cols, drop_first=False)

# Ensure X_test has the same one-hot encoded columns (same order) as X_train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

print("Train/Test shapes:", X_train.shape, X_test.shape)
print("Fraud rate in test set:", round(y_test.mean() * 100, 3), "%")

Train/Test shapes: (12335, 100) (3084, 100)
Fraud rate in test set: 5.999 %


In [ ]:
def train_xgb(X_tr, y_tr, sample_weight=None, scale_pos_weight=None):
    """
    Trains a basic XGBoost model.
    - scale_pos_weight helps in imbalanced learning (baseline)
    - sample_weight used for cost-sensitive training
    """
    model = XGBClassifier(

        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        random_state=42,
        eval_metric="logloss",
        n_jobs=-1,

        # Imbalance handling (baseline)
        scale_pos_weight=scale_pos_weight
    )

    # Train normally or with cost-sensitive sample weights
    if sample_weight is None:
        model.fit(X_tr, y_tr)
    else:
        model.fit(X_tr, y_tr, sample_weight=sample_weight)

    return model


def threshold_table(y_true, y_score, start_q=0.99, end_q=0.80, step=0.01):
    """
    Evaluate thresholds that correspond to investigating top 1%...top 20%.
    Quantile 0.99 ≈ top 1% highest scores.
    """
    rows = []

    for q in np.arange(start_q, end_q, -step):  # 0.99, 0.98, ..., 0.81
        thr = float(np.quantile(y_score, q))
        y_pred = (y_score >= thr).astype(int)

        rows.append({
            "quantile": round(float(q), 2),
            "threshold": thr,
            "alerts_%": float(y_pred.mean() * 100),
            "recall": float(recall_score(y_true, y_pred, zero_division=0)),
            "precision": float(precision_score(y_true, y_pred, zero_division=0)),
            "f1_score": float(f1_score(y_true, y_pred, zero_division=0)),
            "fraud_captured_%": float((y_true[y_pred == 1].sum() / y_true.sum()) * 100) if y_true.sum() > 0 else 0.0
        })

    out = pd.DataFrame(rows).sort_values("threshold", ascending=False)
    return out


In [ ]:
def cost_scenario_table(y_true, y_score, fn_cost, fp_costs, quantiles):
    """
    Evaluate cost under different FP cost assumptions, using top-k quantiles.
    total_cost = FN * fn_cost + FP * fp_cost
    """
    rows = []
    for fp_cost in fp_costs:
        for q in quantiles:
            thr = float(np.quantile(y_score, q))
            y_pred = (y_score >= thr).astype(int)

            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
            total_cost = (fn * fn_cost) + (fp * fp_cost)

            rows.append({
                "fp_cost": fp_cost,
                "quantile": q,
                "threshold": thr,
                "alerts_%": float(y_pred.mean() * 100),
                "recall": float(tp / (tp + fn)) if (tp + fn) > 0 else 0.0,
                "precision": float(tp / (tp + fp)) if (tp + fp) > 0 else 0.0,
                "total_cost": float(total_cost),
                "fp": int(fp),
                "fn": int(fn)
            })

    scenario_df = pd.DataFrame(rows).sort_values(["fp_cost", "quantile"], ascending=[True, False])
    return scenario_df

In [ ]:
# (A) Baseline (no SMOTE): use scale_pos_weight
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
xgb_base = train_xgb(X_train, y_train, scale_pos_weight=scale_pos_weight)

# (B) SMOTE
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
xgb_smote = train_xgb(X_train_sm, y_train_sm)


In [ ]:
FP_COST = 500
FN_COST = 15000
POS_WEIGHT = FN_COST / FP_COST  # 30.0

sample_weight = np.where(y_train_sm == 1, POS_WEIGHT, 1.0)
xgb_smote_cost = train_xgb(X_train_sm, y_train_sm, sample_weight=sample_weight)


In [ ]:
y_score_base = xgb_base.predict_proba(X_test)[:, 1]
y_score_smote = xgb_smote.predict_proba(X_test)[:, 1]
y_score_smote_cost = xgb_smote_cost.predict_proba(X_test)[:, 1]

print("\nROC-AUC")
print("Baseline:", round(roc_auc_score(y_test, y_score_base), 4))
print("SMOTE:", round(roc_auc_score(y_test, y_score_smote), 4))
print("SMOTE + Cost-Sensitive:", round(roc_auc_score(y_test, y_score_smote_cost), 4))



ROC-AUC
Baseline: 0.8012
SMOTE: 0.8168
SMOTE + Cost-Sensitive: 0.7914


In [ ]:
print("\nThreshold Analysis (Baseline)")
op_base = threshold_table(y_test.values, y_score_base)
display(op_base)

print("\nThreshold Analysis (SMOTE)")
op_smote = threshold_table(y_test.values, y_score_smote)
display(op_smote)

print("\nThreshold Analysis (SMOTE + Cost-Sensitive)")
op_smote_cost = threshold_table(y_test.values, y_score_smote_cost)
display(op_smote_cost)




Threshold Analysis (Baseline)


,quantile,threshold,alerts_%,recall,precision,f1_score,fraud_captured_%
0,0.99,0.835399,1.005188,0.091892,0.548387,0.157407,9.189189
1,0.98,0.788857,2.010376,0.124324,0.370968,0.186235,12.432432
2,0.97,0.762634,3.015564,0.145946,0.290323,0.194245,14.594595
3,0.96,0.736885,4.020752,0.156757,0.233871,0.187702,15.675676
4,0.95,0.712555,5.025940,0.194595,0.232258,0.211765,19.459459
5,0.94,0.689676,5.998703,0.210811,0.210811,0.210811,21.081081
6,0.93,0.664327,7.003891,0.248649,0.212963,0.229426,24.864865
7,0.92,0.647966,8.009079,0.275676,0.206478,0.236111,27.567568
8,0.91,0.632345,9.014267,0.291892,0.194245,0.233261,29.189189
9,0.90,0.612643,10.019455,0.318919,0.190939,0.238866,31.891892



Threshold Analysis (SMOTE)


,quantile,threshold,alerts_%,recall,precision,f1_score,fraud_captured_%
0,0.99,0.483868,1.005188,0.086486,0.516129,0.148148,8.648649
1,0.98,0.391328,2.010376,0.129730,0.387097,0.194332,12.972973
2,0.97,0.335108,3.015564,0.162162,0.322581,0.215827,16.216216
3,0.96,0.300967,4.020752,0.172973,0.258065,0.207120,17.297297
4,0.95,0.279515,5.025940,0.194595,0.232258,0.211765,19.459459
5,0.94,0.255381,5.998703,0.232432,0.232432,0.232432,23.243243
6,0.93,0.232642,7.003891,0.259459,0.222222,0.239401,25.945946
7,0.92,0.212956,8.009079,0.286486,0.214575,0.245370,28.648649
8,0.91,0.200923,9.014267,0.297297,0.197842,0.237581,29.729730
9,0.90,0.190586,10.019455,0.324324,0.194175,0.242915,32.432432



Threshold Analysis (SMOTE + Cost-Sensitive)


,quantile,threshold,alerts_%,recall,precision,f1_score,fraud_captured_%
0,0.99,0.937986,1.005188,0.086486,0.516129,0.148148,8.648649
1,0.98,0.888864,2.010376,0.118919,0.354839,0.178138,11.891892
2,0.97,0.860556,3.015564,0.156757,0.311828,0.208633,15.675676
3,0.96,0.819817,4.020752,0.200000,0.298387,0.239482,20.000000
4,0.95,0.795291,5.025940,0.232432,0.277419,0.252941,23.243243
5,0.94,0.766176,5.998703,0.243243,0.243243,0.243243,24.324324
6,0.93,0.732602,7.003891,0.281081,0.240741,0.259352,28.108108
7,0.92,0.700977,8.009079,0.291892,0.218623,0.250000,29.189189
8,0.91,0.662803,9.014267,0.302703,0.201439,0.241901,30.270270
9,0.90,0.642941,10.019455,0.335135,0.200647,0.251012,33.513514


In [ ]:
# Different investigation costs (FP)
FP_COSTS = [250, 500, 1000]
# Choose a few realistic investigation budgets
QUANTILES = [0.99, 0.97, 0.95, 0.90, 0.85]

print("\nCost Scenario (Baseline)")
scenario_base = cost_scenario_table(y_test.values, y_score_base, fn_cost=FN_COST, fp_costs=FP_COSTS, quantiles=QUANTILES)
display(scenario_base)

print("\nCost Scenario (SMOTE)")
scenario_smote = cost_scenario_table(y_test.values, y_score_smote, fn_cost=FN_COST, fp_costs=FP_COSTS, quantiles=QUANTILES)
display(scenario_smote)

print("\nCost Scenario (SMOTE + Cost-Sensitive)")
scenario_smote_cost = cost_scenario_table(y_test.values, y_score_smote_cost, fn_cost=FN_COST, fp_costs=FP_COSTS, quantiles=QUANTILES)
display(scenario_smote_cost)


Cost Scenario (Baseline)


,fp_cost,quantile,threshold,alerts_%,recall,precision,total_cost,fp,fn
0,250,0.99,0.835399,1.005188,0.091892,0.548387,2523500.0,14,168
1,250,0.97,0.762634,3.015564,0.145946,0.290323,2386500.0,66,158
2,250,0.95,0.712555,5.025940,0.194595,0.232258,2264750.0,119,149
3,250,0.90,0.612643,10.019455,0.318919,0.190939,1952500.0,250,126
4,250,0.85,0.527164,15.012970,0.416216,0.166307,1716500.0,386,108
5,500,0.99,0.835399,1.005188,0.091892,0.548387,2527000.0,14,168
6,500,0.97,0.762634,3.015564,0.145946,0.290323,2403000.0,66,158
7,500,0.95,0.712555,5.025940,0.194595,0.232258,2294500.0,119,149
8,500,0.90,0.612643,10.019455,0.318919,0.190939,2015000.0,250,126
9,500,0.85,0.527164,15.012970,0.416216,0.166307,1813000.0,386,108



Cost Scenario (SMOTE)


,fp_cost,quantile,threshold,alerts_%,recall,precision,total_cost,fp,fn
0,250,0.99,0.483868,1.005188,0.086486,0.516129,2538750.0,15,169
1,250,0.97,0.335108,3.015564,0.162162,0.322581,2340750.0,63,155
2,250,0.95,0.279515,5.025940,0.194595,0.232258,2264750.0,119,149
3,250,0.90,0.190586,10.019455,0.324324,0.194175,1937250.0,249,125
4,250,0.85,0.138967,15.012970,0.427027,0.170626,1686000.0,384,106
5,500,0.99,0.483868,1.005188,0.086486,0.516129,2542500.0,15,169
6,500,0.97,0.335108,3.015564,0.162162,0.322581,2356500.0,63,155
7,500,0.95,0.279515,5.025940,0.194595,0.232258,2294500.0,119,149
8,500,0.90,0.190586,10.019455,0.324324,0.194175,1999500.0,249,125
9,500,0.85,0.138967,15.012970,0.427027,0.170626,1782000.0,384,106



Cost Scenario (SMOTE + Cost-Sensitive)


,fp_cost,quantile,threshold,alerts_%,recall,precision,total_cost,fp,fn
0,250,0.99,0.937986,1.005188,0.086486,0.516129,2538750.0,15,169
1,250,0.97,0.860556,3.015564,0.156757,0.311828,2356000.0,64,156
2,250,0.95,0.795291,5.025940,0.232432,0.277419,2158000.0,112,142
3,250,0.90,0.642941,10.019455,0.335135,0.200647,1906750.0,247,123
4,250,0.85,0.491605,15.012970,0.448649,0.179266,1625000.0,380,102
5,500,0.99,0.937986,1.005188,0.086486,0.516129,2542500.0,15,169
6,500,0.97,0.860556,3.015564,0.156757,0.311828,2372000.0,64,156
7,500,0.95,0.795291,5.025940,0.232432,0.277419,2186000.0,112,142
8,500,0.90,0.642941,10.019455,0.335135,0.200647,1968500.0,247,123
9,500,0.85,0.491605,15.012970,0.448649,0.179266,1720000.0,380,102


In [ ]:
def best_by_cost(scenario_df, fp_cost_value):
    sub = scenario_df[scenario_df["fp_cost"] == fp_cost_value].sort_values("total_cost")
    return sub.iloc[0]

print("\nBest operating point (lowest cost) assuming FP_COST = 500")
best_base = best_by_cost(scenario_base, 500)
best_smote = best_by_cost(scenario_smote, 500)
best_smote_cost = best_by_cost(scenario_smote_cost, 500)

summary = pd.DataFrame([
    {"model": "Baseline", "quantile": best_base["quantile"], "alerts_%": best_base["alerts_%"], "recall": best_base["recall"], "precision": best_base["precision"], "total_cost": best_base["total_cost"]},
    {"model": "SMOTE", "quantile": best_smote["quantile"], "alerts_%": best_smote["alerts_%"], "recall": best_smote["recall"], "precision": best_smote["precision"], "total_cost": best_smote["total_cost"]},
    {"model": "SMOTE + Cost-Sensitive", "quantile": best_smote_cost["quantile"], "alerts_%": best_smote_cost["alerts_%"], "recall": best_smote_cost["recall"], "precision": best_smote_cost["precision"], "total_cost": best_smote_cost["total_cost"]},
]).sort_values("total_cost")

display(summary)


Best operating point (lowest cost) assuming FP_COST = 500


,model,quantile,alerts_%,recall,precision,total_cost
2,SMOTE + Cost-Sensitive,0.85,15.01297,0.448649,0.179266,1720000.0
1,SMOTE,0.85,15.01297,0.427027,0.170626,1782000.0
0,Baseline,0.85,15.01297,0.416216,0.166307,1813000.0


In [ ]:
## CatBoost

In [ ]:
!pip -q install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.3 MB/s eta 0:00:00


In [ ]:

from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score



df_cb = pd.read_csv('/content/drive/MyDrive/final_year_project/fraud_oracle 4.csv')


# Target / features
y = df_cb["FraudFound_P"]
X = df_cb.drop(columns=["FraudFound_P"])


cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

# Convert to string (CatBoost can handle string categoricals well)
for c in cat_cols:
    X[c] = X[c].astype(str)

# Categorical indices
cat_feature_indices = [X.columns.get_loc(c) for c in cat_cols]
print("Categorical columns for CatBoost:", cat_cols)


# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# Class weights baseline
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print("pos_weight:", pos_weight)


# Train baseline CatBoost

cb = CatBoostClassifier(
    iterations=600,
    learning_rate=0.05,
    depth=6,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=200,
    class_weights=[1, pos_weight],
)

cb.fit(
    X_train, y_train,
    cat_features=cat_feature_indices,
    eval_set=(X_test, y_test),
    use_best_model=True
)


# Evaluate at threshold 0.5
y_prob = cb.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.5).astype(int)

print("\n=== CatBoost baseline (thr=0.5) ===")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))


# Threshold sweep

rows = []
for thr in np.arange(0.05, 0.60, 0.05):
    pred_thr = (y_prob >= thr).astype(int)
    rows.append({
        "threshold": round(thr, 2),
        "precision": precision_score(y_test, pred_thr, zero_division=0),
        "recall": recall_score(y_test, pred_thr),
        "f1": f1_score(y_test, pred_thr),
        "alerts_%": pred_thr.mean() * 100
    })

thr_df = pd.DataFrame(rows)
print("\nThreshold analysis (CatBoost):")
thr_df


Categorical columns for CatBoost: ['Month', 'DayOfWeek', 'Make', 'AccidentArea', 'DayOfWeekClaimed', 'MonthClaimed', 'Sex', 'MaritalStatus', 'Fault', 'PolicyType', 'VehicleCategory', 'VehiclePrice', 'Days_Policy_Accident', 'Days_Policy_Claim', 'PastNumberOfClaims', 'AgeOfVehicle', 'AgeOfPolicyHolder', 'PoliceReportFiled', 'WitnessPresent', 'AgentType', 'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars', 'BasePolicy']
pos_weight: 15.715447154471544
0:	test: 0.7680039	best: 0.7680039 (0)	total: 48.5ms	remaining: 29.1s
200:	test: 0.9296253	best: 0.9296868 (197)	total: 12.7s	remaining: 25.2s
400:	test: 0.9682780	best: 0.9682780 (400)	total: 28s	remaining: 13.9s
599:	test: 0.9773286	best: 0.9773696 (595)	total: 43.3s	remaining: 0us

bestTest = 0.9773696428
bestIteration = 595

Shrink model to first 596 iterations.

=== CatBoost baseline (thr=0.5) ===
Confusion Matrix:
 [[2680  219]
 [  10  175]]
              precision    recall  f1-score   support

           0     0.9963    0.92

,threshold,precision,recall,f1,alerts_%
0,0.05,0.144193,1.000000,0.252044,41.601816
1,0.10,0.186047,0.994595,0.313458,32.068742
2,0.15,0.219952,0.989189,0.359882,26.977951
3,0.20,0.257790,0.983784,0.408530,22.892348
4,0.25,0.287066,0.983784,0.444444,20.557717
5,0.30,0.311532,0.978378,0.472585,18.839170
6,0.35,0.341509,0.978378,0.506294,17.185473
7,0.40,0.383475,0.978378,0.550989,15.304799
8,0.45,0.407745,0.967568,0.573718,14.234760
9,0.50,0.444162,0.945946,0.604491,12.775616
